In [61]:
#documentation: https://librosa.github.io/librosa/generated/librosa.feature.mfcc.html

from python_speech_features import mfcc
from python_speech_features import logfbank
import scipy.io.wavfile as wav
import glob
import numpy as np

In [62]:
import os
cwd = os.getcwd()
#cwd = "C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator"
print(cwd)

C:\Users\qan96\Documents\GitHub\Audio-Calculator


In [84]:
def join_features(mfcc, fbank):
    features = np.concatenate((mfcc, fbank), axis=1)
    return features

plus = 11
minus = 12
times = 13
over = 14

features = []
labels = []

def read_file(files, label):
    for file in files:
        (rate,sig) = wav.read(file)
        mfcc_feat = mfcc(sig, rate, nfft=1536)
        fbank_feat = logfbank(sig, rate, nfft=1536)
        acoustic_features = join_features(mfcc_feat, fbank_feat)
        features.append(acoustic_features)
        labels.append(label)
        
feature_types = ["zero", "one", "two", "three", "four", "five", "six", "seven", "eight",\
                 "nine", "plus", "minus", "times", "over"]

for i in feature_types:
    os.chdir(cwd+"\\recording data\\" + i + "\\")
    files = glob.glob( os.path.join(os.getcwd(), '*.wav') )
    read_file(files, i)

[array([[16.26976594,  6.75409837,  9.39135608, ..., 10.72678368,
        11.60265751, 12.26555555],
       [17.28148868,  7.17576756, 12.95945248, ..., 11.03262235,
        12.23517096, 12.61697634],
       [17.61772288,  5.64719229, 21.7397422 , ..., 11.68143268,
        12.53069515, 12.87645569],
       ...,
       [13.25928728, 13.48520309, -5.18970101, ...,  6.92544961,
         7.63741251,  7.53024637],
       [12.63030516, 10.94916611, -4.51168871, ...,  6.87031632,
         7.31165291,  7.53166084],
       [12.06383846, 11.25537577,  0.50075832, ...,  6.22250767,
         6.92430493,  7.55139646]])]
['zero']
